In [ ]:
!pip install numba

import numpy as np
import urllib.request
from numba import cuda
import time


In [ ]:
# 1. Wczytanie danych ch150.tsp z TSPLIB

def load_ch150():
    """
    Pobiera plik ch150.tsp z TSPLIB i zwraca:
    - coords: tablica numpy (n, 2) z współrzędnymi [x, y]
    """
    url = "https://raw.githubusercontent.com/mastqe/tsplib/master/ch150.tsp"

    with urllib.request.urlopen(url) as f:
        lines = f.read().decode("utf-8").strip().splitlines()

    coords = []
    node_section = False

    for line in lines:
        line = line.strip()
        if line == "NODE_COORD_SECTION":
            node_section = True
            continue
        if line == "EOF":
            break
        if node_section:
            parts = line.split()
            if len(parts) >= 3:
                _id = int(parts[0])
                x = float(parts[1])
                y = float(parts[2])
                coords.append((x, y))

    return np.array(coords, dtype=np.float32)

coords = load_ch150()
n_cities = coords.shape[0]
print("Liczba miast:", n_cities)

# 2. Macierz odległości EUC_2D

def build_distance_matrix(coords):
    diff = coords[:, np.newaxis, :] - coords[np.newaxis, :, :]
    dist = np.sqrt(np.sum(diff ** 2, axis=2))
    return np.rint(dist).astype(np.int32)

dist_matrix = build_distance_matrix(coords)
print("dist_matrix shape:", dist_matrix.shape)


Liczba miast: 150
dist_matrix shape: (150, 150)


In [ ]:
# 3. GA – operatory CPU

def random_individual(n):
    perm = np.arange(n, dtype=np.int32)
    np.random.shuffle(perm)
    return perm

def tour_length_cpu(individual, dist_matrix):
    total = 0
    n = len(individual)
    prev = individual[0]
    for i in range(1, n):
        city = individual[i]
        total += dist_matrix[prev, city]
        prev = city
    total += dist_matrix[prev, individual[0]]
    return total

def tournament_selection(population, fitness, k=3):
    idxs = np.random.choice(len(population), size=k, replace=False)
    best_idx = idxs[np.argmin(fitness[idxs])]
    return population[best_idx]

def order_crossover(p1, p2):
    n = len(p1)
    a, b = sorted(np.random.randint(0, n, size=2))
    child = np.full(n, -1, dtype=np.int32)

    child[a:b+1] = p1[a:b+1]

    pos = (b + 1) % n
    for gene in p2:
        if gene not in child:
            child[pos] = gene
            pos = (pos + 1) % n
    return child

def mutate_swap(individual):
    n = len(individual)
    i, j = np.random.randint(0, n, size=2)
    individual[i], individual[j] = individual[j], individual[i]


In [ ]:
# 4. Kernel CUDA – liczenie długości tras populacji

@cuda.jit
def evaluate_population_kernel(population, dist_matrix, fitness):
    idx = cuda.grid(1)  # globalny indeks wątku
    n_pop = population.shape[0]
    n_cities = population.shape[1]

    if idx < n_pop:
        total = 0
        # pierwszy punkt
        prev = population[idx, 0]
        # przejścia między kolejnymi miastami
        for i in range(1, n_cities):
            city = population[idx, i]
            total += dist_matrix[prev, city]
            prev = city
        # powrót do startu
        total += dist_matrix[prev, population[idx, 0]]
        fitness[idx] = total


In [ ]:
# 5. Funkcje pomocnicze: ewaluacja na GPU i na CPU

def evaluate_population_gpu(population, dist_matrix):
    """
    population: numpy (pop_size, n_cities), int32
    dist_matrix: numpy (n_cities, n_cities), int32
    Zwraca: fitness jako numpy (pop_size,), int32
    """
    pop_size, n_cities = population.shape

    d_pop = cuda.to_device(population)
    d_dist = cuda.to_device(dist_matrix)
    d_fitness = cuda.device_array(pop_size, dtype=np.int32)

    threads_per_block = 128
    blocks_per_grid = (pop_size + threads_per_block - 1) // threads_per_block

    evaluate_population_kernel[blocks_per_grid, threads_per_block](d_pop, d_dist, d_fitness)

    fitness = d_fitness.copy_to_host()
    return fitness

def evaluate_population_cpu(population, dist_matrix):
    fitness = np.empty(len(population), dtype=np.int32)
    for i, ind in enumerate(population):
        fitness[i] = tour_length_cpu(ind, dist_matrix)
    return fitness


In [ ]:
# ============================================
# 7. Helper: krzyżowanie + mutacja na CPU (dla porównania)
# ============================================

def order_crossover_cpu(p1, p2, a, b):
    """
    OX na CPU z zadanym segmentem [a,b].
    """
    n = len(p1)
    child = np.full(n, -1, dtype=np.int32)
    child[a:b+1] = p1[a:b+1]

    pos = (b + 1) % n
    for gene in p2:
        if gene not in child:
            while child[pos] != -1:
                pos = (pos + 1) % n
            child[pos] = gene
    return child

def crossover_mutate_cpu(population, parent1_idx, parent2_idx,
                         crossover_rate, mutation_rate):
    """
    Wersja CPU krzyżowania + mutacji dla wszystkich par.
    Zwraca tablicę dzieci (2*num_pairs, n_cities).
    """
    num_pairs = len(parent1_idx)
    n_cities = population.shape[1]
    children = np.empty((2 * num_pairs, n_cities), dtype=np.int32)

    for pair_id in range(num_pairs):
        p1 = population[parent1_idx[pair_id]]
        p2 = population[parent2_idx[pair_id]]

        if np.random.rand() < crossover_rate:
            a, b = sorted(np.random.randint(0, n_cities, size=2))
            c1 = order_crossover_cpu(p1, p2, a, b)
            c2 = order_crossover_cpu(p2, p1, a, b)
        else:
            c1 = p1.copy()
            c2 = p2.copy()

        # mutacja
        if np.random.rand() < mutation_rate:
            i, j = np.random.randint(0, n_cities, size=2)
            c1[i], c1[j] = c1[j], c1[i]
        if np.random.rand() < mutation_rate:
            i, j = np.random.randint(0, n_cities, size=2)
            c2[i], c2[j] = c2[j], c2[i]

        children[2 * pair_id] = c1
        children[2 * pair_id + 1] = c2

    return children

# ============================================
# 8. Helper: krzyżowanie + mutacja na GPU
# ============================================

def crossover_mutate_gpu(population, parent1_idx, parent2_idx,
                         crossover_rate, mutation_rate):
    """
    Przygotowuje dane, odpala kernel na GPU i zwraca dzieci (2*num_pairs, n_cities).
    """
    num_pairs = len(parent1_idx)
    n_cities = population.shape[1]

    parent1_idx = np.array(parent1_idx, dtype=np.int32)
    parent2_idx = np.array(parent2_idx, dtype=np.int32)

    # Losujemy segmenty krzyżowania i indeksy mutacji na CPU
    cross_start = np.empty(num_pairs, dtype=np.int32)
    cross_end = np.empty(num_pairs, dtype=np.int32)

    # Uproszczenie: jeśli nie ma krzyżowania, ustaw a=0,b=-1 -> kernel wstawi tylko mutacje
    for i in range(num_pairs):
        if np.random.rand() < crossover_rate:
            a, b = sorted(np.random.randint(0, n_cities, size=2))
        else:
            a, b = 0, -1  # specjalny przypadek: brak krzyżowania
        cross_start[i] = a
        cross_end[i] = b

    mut_i = np.empty(2 * num_pairs, dtype=np.int32)
    mut_j = np.empty(2 * num_pairs, dtype=np.int32)
    for k in range(2 * num_pairs):
        if np.random.rand() < mutation_rate:
            i1, j1 = np.random.randint(0, n_cities, size=2)
        else:
            i1, j1 = 0, 0  # "mutacja zerowa" (nic nie zmienia)
        mut_i[k] = i1
        mut_j[k] = j1

    # Kopiujemy dane na GPU
    d_population = cuda.to_device(population)
    d_parent1_idx = cuda.to_device(parent1_idx)
    d_parent2_idx = cuda.to_device(parent2_idx)
    d_cross_start = cuda.to_device(cross_start)
    d_cross_end = cuda.to_device(cross_end)
    d_mut_i = cuda.to_device(mut_i)
    d_mut_j = cuda.to_device(mut_j)
    d_children = cuda.device_array((2 * num_pairs, n_cities), dtype=np.int32)

    # Konfiguracja siatki
    threads_per_block = 128
    blocks_per_grid = (num_pairs + threads_per_block - 1) // threads_per_block

    # Uruchom kernel
    crossover_mutate_kernel[blocks_per_grid, threads_per_block](
        d_population,
        d_parent1_idx, d_parent2_idx,
        d_cross_start, d_cross_end,
        d_mut_i, d_mut_j,
        d_children
    )

    children = d_children.copy_to_host()
    return children


In [ ]:
# 6. Główna pętla GA – z wyborem CPU / GPU dla oceny populacji

def genetic_algorithm_tsp(
    dist_matrix,
    n_generations=500,
    population_size=200,
    crossover_rate=0.9,
    mutation_rate=0.2,
    tournament_k=3,
    elitism=True,
    use_gpu=True
):
    n = dist_matrix.shape[0]

    # inicjalizacja
    population = np.array([random_individual(n) for _ in range(population_size)], dtype=np.int32)
    best_cost_history = []
    best_indiv_history = []

    for gen in range(n_generations):
        # --- ewaluacja populacji ---
        if use_gpu:
            fitness = evaluate_population_gpu(population, dist_matrix)
        else:
            fitness = evaluate_population_cpu(population, dist_matrix)

        # najlepszy osobnik
        best_idx = np.argmin(fitness)
        best_ind = population[best_idx].copy()
        best_cost = int(fitness[best_idx])

        best_cost_history.append(best_cost)
        best_indiv_history.append(best_ind)

        if gen % 50 == 0 or gen == n_generations - 1:
            mode = "GPU" if use_gpu else "CPU"
            print(f"[{mode}] Pokolenie {gen:4d} | najlepszy koszt: {best_cost}")

        # --- tworzenie nowej populacji ---
        new_population = []

        # elita
        if elitism:
            new_population.append(best_ind)

        # reszta populacji
        while len(new_population) < population_size:
            p1 = tournament_selection(population, fitness, k=tournament_k)
            p2 = tournament_selection(population, fitness, k=tournament_k)

            if np.random.rand() < crossover_rate:
                c1 = order_crossover(p1, p2)
                c2 = order_crossover(p2, p1)
            else:
                c1 = p1.copy()
                c2 = p2.copy()

            if np.random.rand() < mutation_rate:
                mutate_swap(c1)
            if np.random.rand() < mutation_rate:
                mutate_swap(c2)

            new_population.append(c1)
            if len(new_population) < population_size:
                new_population.append(c2)

        population = np.array(new_population, dtype=np.int32)

    best_overall_idx = np.argmin(best_cost_history)
    best_overall_cost = best_cost_history[best_overall_idx]
    best_overall_ind = best_indiv_history[best_overall_idx]

    return best_overall_ind, best_overall_cost, best_cost_history


In [ ]:
# 7. Pomiar czasu: CPU vs GPU

def run_and_time(use_gpu, label, **ga_kwargs):
    np.random.seed(42)

    start = time.perf_counter()
    best_tour, best_cost, history = genetic_algorithm_tsp(
        dist_matrix,
        use_gpu=use_gpu,
        **ga_kwargs
    )
    end = time.perf_counter()
    elapsed = end - start

    print(f"\n[{label}] czas: {elapsed:.3f} s | najlepszy koszt: {best_cost}")
    return elapsed, best_cost, history, best_tour

ga_params = dict(
    n_generations=300,
    population_size=512,  # warto trochę zwiększyć populację, żeby GPU miało co robić
    crossover_rate=0.9,
    mutation_rate=0.2,
    tournament_k=3,
    elitism=True
)

print("=== Porównanie CPU vs GPU (ocena populacji) ===")
t_cpu, c_cpu, h_cpu, tour_cpu = run_and_time(False, "CPU", **ga_params)
t_gpu, c_gpu, h_gpu, tour_gpu = run_and_time(True, "GPU", **ga_params)

speedup = t_cpu / t_gpu if t_gpu > 0 else float("inf")
print(f"\nPrzyspieszenie (speedup) = czas_CPU / czas_GPU = {speedup:.3f}")


=== Porównanie CPU vs GPU (ocena populacji) ===
[CPU] Pokolenie    0 | najlepszy koszt: 48316
[CPU] Pokolenie   50 | najlepszy koszt: 33399
[CPU] Pokolenie  100 | najlepszy koszt: 29671
[CPU] Pokolenie  150 | najlepszy koszt: 26971
[CPU] Pokolenie  200 | najlepszy koszt: 25494
[CPU] Pokolenie  250 | najlepszy koszt: 23537
[CPU] Pokolenie  299 | najlepszy koszt: 23399

[CPU] czas: 107.033 s | najlepszy koszt: 23399


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[GPU] Pokolenie    0 | najlepszy koszt: 48316
[GPU] Pokolenie   50 | najlepszy koszt: 33399
[GPU] Pokolenie  100 | najlepszy koszt: 29671
[GPU] Pokolenie  150 | najlepszy koszt: 26971
[GPU] Pokolenie  200 | najlepszy koszt: 25494
[GPU] Pokolenie  250 | najlepszy koszt: 23537
[GPU] Pokolenie  299 | najlepszy koszt: 23399

[GPU] czas: 90.881 s | najlepszy koszt: 23399

Przyspieszenie (speedup) = czas_CPU / czas_GPU = 1.178
